In [1]:
import pylab
import scipy.signal as signal
import numpy as np
import numpy
from scipy.signal import detrend
import os
import h5py

## This file shows codes used for generation of ECG samples

In [ ]:
mixed_vectors = []
maternal_vectors = []
fetal_vectors = []
ft_mixed = []
#real_ft_mixed = []
#im_ft_mixed = []
t_vector = range(TRAIN_SIZE)
ft_shape = (258, 33)

In [ ]:
#### Generating the Mixed ECG Signal ####

def generate_mecg(maternal_bpm, fetal_bpm):
    maternal_ecg_v = signalz.ecgsyn(hrmean=maternal_bpm, n=40)[0]
    fetal_ecg_v = signalz.ecgsyn(hrmean=fetal_bpm, n=80)[0]
    
    print(maternal_ecg_v.shape)
    
    starting_vector = np.random.randint(0,high=maternal_ecg_v.shape[0]-4096)
    ending_vector = starting_vector + 4096
    
    maternal_ecg_v = maternal_ecg_v[starting_vector:ending_vector]
    fetal_ecg_v = fetal_ecg_v[starting_vector:ending_vector]
    
    return [range(4096), 100*(maternal_ecg_v+0.6*fetal_ecg_v), 200*maternal_ecg_v, 200*fetal_ecg_v*0.6]

def sample_bpm(low, up, num_samples=200):
    return np.random.uniform(low,up,num_samples)

def short_time_fourier_transform(v):
    ft = np.array(signal.stft(v,fs=500)[2])
    r_ft = np.zeros(ft.shape)
    i_ft = np.zeros(ft.shape)
    for j in range(len(r_ft)):
        for k in range(len(r_ft[j])):
            r_ft[j][k] = ft[j][k].real
            i_ft[j][k] = ft[j][k].imag
    ft = np.concatenate((r_ft,i_ft))
    return (np.reshape(ft,ft.shape[0]*ft.shape[1]), ft, r_ft, i_ft)



def generate_training_samples(num_samples, train_size):
    

    # As cardiac output increases, 
    # the heart rate at rest speeds up from a normal prepregnancy rate of about 70 beats per minute to 80 or 90 beats per minute

    # normal fetal heart rate for mid-pregnancy to about 120 to 180 bpm.

    m_bpm = sample_bpm(80,90,num_samples)
    f_bpm = sample_bpm(120,180,num_samples)

    for i in range(num_samples):
        print("Generating Sample ", i, m_bpm[i], f_bpm[i])
        mecg_t, mecg_v, maternal_ecg_v, fetal_ecg_v = generate_mecg(m_bpm[i],f_bpm[i])
        #starting_index = np.random.randint(0,high=(maternal_ecg_v.shape[0]-train_size))
        starting_index = 0
        ending_index = starting_index+train_size        
        a, b, c, d = short_time_fourier_transform(mecg_v[starting_index:ending_index])
        
        if (i == 0):
            
            pylab.plot(mecg_t[0:train_size],mecg_v[starting_index:ending_index])
            pylab.xlabel('Sample number')
            pylab.ylabel('Amplitude (normalised)')
            pylab.title('Heart Mixed ECG Template with maternal bpm %d and fetal bpm %d' %(m_bpm[i], f_bpm[i]))
            pylab.show()

            pylab.plot(mecg_t[0:train_size],maternal_ecg_v[starting_index:ending_index])
            pylab.xlabel('Sample number')
            pylab.ylabel('Amplitude (normalised)')
            pylab.title('Heart Maternal ECG Template with maternal bpm %d and fetal bpm %d' %(m_bpm[i], f_bpm[i]))
            pylab.show()

            pylab.plot(mecg_t[0:train_size],fetal_ecg_v[starting_index:ending_index])
            pylab.xlabel('Sample number')
            pylab.ylabel('Amplitude (normalised)')
            pylab.title('Heart Fetal ECG Template with maternal bpm %d and fetal bpm %d' %(m_bpm[i], f_bpm[i]))
            pylab.show()
        
        mixed_vectors.append(mecg_v[starting_index:ending_index])
        maternal_vectors.append(maternal_ecg_v[starting_index:ending_index])
        fetal_vectors.append(fetal_ecg_v[starting_index:ending_index])
        ft_mixed.append(a)
            
        display.clear_output()
        
    return 

In [ ]:
## Make samples
TRAIN_AMT = 2000
generate_training_samples(TRAIN_AMT, TRAIN_SIZE)

In [ ]:
train = (np.array(mixed_vectors), 
        np.array(maternal_vectors), 
        np.array(fetal_vectors), 
        np.array(ft_mixed), 
        ft_shape, 
        ft_shape, 
        t_vector, 
        ft_shape)

## print some tests:
for i in range(4):
    print(train[i].shape)
print(train[7])

In [ ]:
FILE_NAME = 'generated_samples.h5'
directory = FILE_NAME
hf = h5py.File(directory, 'w')

hf.create_dataset('mixed_vectors', data=train[0])
hf.create_dataset('maternal_vectors', data=train[1])
hf.create_dataset('fetal_vectors', data=train[2])
hf.create_dataset('reshaped_ft_mixed', data=train[3])
hf.create_dataset('m_bpm', data=train[4])
hf.create_dataset('f_bpm', data=train[5])
hf.create_dataset('t_vector', data=train[6])
hf.create_dataset('ft_shape', data=train[7])
    
hf.close()

In [ ]:
# Load Sample
FILE_NAME = 'generated_samples.h5'
directory = FILE_NAME
hf = h5py.File(directory, 'r')
train_temp = []

train_temp.append(hf.get('mixed_vectors'))
train_temp.append(hf.get('maternal_vectors'))
train_temp.append(hf.get('fetal_vectors'))
train_temp.append(hf.get('reshaped_ft_mixed'))
train_temp.append(hf.get('m_bpm'))
train_temp.append(hf.get('f_bpm'))
train_temp.append(hf.get('t_vector'))
train_temp.append(hf.get('ft_shape'))

train = []

## print some tests:
for i in range(7):
    print(train_temp[i].shape)
print(train_temp[7])

for i in range(NUM_SAMPLES):
    result = {"maternal_bpm": train_temp[4],
              "fetal_bpm": train_temp[5],
              "t_vector": np.array(train_temp[6]), 
              "v_vector": train_temp[0][i],
              "m_v_vector": train_temp[1][i],
              "f_v_vector": train_temp[2][i],
              "stft": train_temp[3][i]
              #"stft_r": np.array(i[8]),
              #"stft_i": np.array(i[9])
             }
    train.append(result)

train = np.array(train)

In [ ]:
print(train[0]["stft"].shape)
print(train.shape)

TRAIN_SHAPE = np.array([258,33])
TRAIN_SIZE = 258*33
print(TRAIN_SHAPE)
print(TRAIN_SIZE)

In [ ]:
def make_noise(size, maternal, fetal):
    result = []
    result1 = []
    result2 = []
    frequencies = 5*np.random.rand(size)
    for i in range(size):
        noise = np.random.randint(10,high=30)*(np.sin(2*np.pi*frequencies[i]*np.array(range(4096))/4096))
        result1.append(maternal[i])
        result2.append(fetal[i])
        temp = (maternal[i]+ noise + fetal[i])/2
        result.append(temp)
    return np.array(result), np.array(result1), np.array(result2)

In [ ]:
mixed_vectors = np.array(train_temp[0])
maternal_vectors = np.array(train_temp[1])
fetal_vectors = np.array(train_temp[2])

#mixed_vectors, maternal_vectors,fetal_vectors = make_noise(2000, maternal_vectors, fetal_vectors)

pylab.plot(mixed_vectors[0])
pylab.show()
pylab.plot(maternal_vectors[0])
pylab.show()
pylab.plot(fetal_vectors[0])
pylab.show()

np.save(ROOT + '\\samples\\' + 'mix_s.npy', mixed_vectors)
np.save(ROOT + '\\samples\\' + 'bass_s.npy', maternal_vectors)
np.save(ROOT + '\\samples\\' + 'sax_s.npy', fetal_vectors)

In [ ]:
print(mixed_vectors.shape)
stft_mixed_vectors = []
stft_maternal_vectors = []
stft_fetal_vectors = []

for i in range(mixed_vectors.shape[0]):
    stft_mixed_vectors.append(short_time_fourier_transform(mixed_vectors[i])[0])
    stft_maternal_vectors.append(short_time_fourier_transform(maternal_vectors[i])[0])
    stft_fetal_vectors.append(short_time_fourier_transform(fetal_vectors[i])[0])
    
np.save(ROOT + '\\samples\\' + 'stft_mix_s.npy', stft_mixed_vectors)
np.save(ROOT + '\\samples\\' + 'stft_bass_s.npy', stft_maternal_vectors)
np.save(ROOT + '\\samples\\' + 'stft_sax_s.npy', stft_fetal_vectors)